In [1]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#import API key
from config import g_key

#access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
#import cities.csv file into a dataframe
cities_df = pd.read_csv("cities.csv")

#get rid of 'City_ID' column
cities_df = cities_df[["City", "Cloudiness", "Country", 
                       "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tasiilaq,100,GL,1579917988,45,65.61,-37.64,23.00,18.34
1,Hrodna,100,BY,1579917989,87,53.69,23.83,35.28,14.54
2,Constitucion,0,CL,1579917942,63,-35.33,-72.42,57.33,5.99
3,Port Alfred,0,ZA,1579917982,94,-33.59,26.89,64.99,1.01
4,Esperance,40,AU,1579917992,57,-33.87,121.90,75.20,3.36


In [3]:
#create heatmap based on humidity

#store latitude and longitude into the variable 'coordinates'
coordinates = cities_df[["Lat", "Lng"]]

#convert humidity to float; all data read from a csv is read as a string
humidity = cities_df["Humidity"].astype(float)

In [4]:
#plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

#add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#create a dataframe of cities that meet specific criteria to me
specs_df = cities_df.loc[(cities_df["Max Temp"] > 80) &
                         (cities_df["Humidity"] < 60) & 
                         (cities_df["Wind Speed"] < 5) | 
                         (cities_df["Wind Speed"] > 30), :].reset_index()

#need to remove newly created index column that appears with reset_index()
specs_df = specs_df[["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
#preview dataframe
specs_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Olafsvik,100,IS,1579918001,89,64.89,-23.71,29.14,37.67
1,Vestmannaeyjar,100,IS,1579918060,82,63.44,-20.27,39.34,35.28
2,Nanortalik,100,GL,1579918320,75,60.14,-45.24,20.73,31.65
3,Vagur,100,FO,1579918354,87,61.47,-6.81,47.16,30.33
4,Loa Janan,3,ID,1579918385,48,-0.58,117.09,91.87,4.34
5,Severo-Kuril'sk,95,RU,1579918391,90,50.68,156.12,17.17,41.03
6,Sibolga,0,ID,1579918432,59,1.74,98.78,87.08,1.45
7,Collie,29,AU,1579918443,31,-33.37,116.15,84.99,4.00
8,Rawson,100,AR,1579918464,47,-43.30,-65.10,70.02,30.87


In [6]:
hotel_df = specs_df.dropna()
hotel_df

#for reference
#to see if there are nan values, code:
#nan_df = specs_df[specs_df.isna().any(axis=1)]
#nan_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Olafsvik,100,IS,1579918001,89,64.89,-23.71,29.14,37.67
1,Vestmannaeyjar,100,IS,1579918060,82,63.44,-20.27,39.34,35.28
2,Nanortalik,100,GL,1579918320,75,60.14,-45.24,20.73,31.65
3,Vagur,100,FO,1579918354,87,61.47,-6.81,47.16,30.33
4,Loa Janan,3,ID,1579918385,48,-0.58,117.09,91.87,4.34
5,Severo-Kuril'sk,95,RU,1579918391,90,50.68,156.12,17.17,41.03
6,Sibolga,0,ID,1579918432,59,1.74,98.78,87.08,1.45
7,Collie,29,AU,1579918443,31,-33.37,116.15,84.99,4.00
8,Rawson,100,AR,1579918464,47,-43.30,-65.10,70.02,30.87


In [7]:
#set up additional columns to hold information
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Olafsvik,100,IS,1579918001,89,64.89,-23.71,29.14,37.67,
1,Vestmannaeyjar,100,IS,1579918060,82,63.44,-20.27,39.34,35.28,
2,Nanortalik,100,GL,1579918320,75,60.14,-45.24,20.73,31.65,
3,Vagur,100,FO,1579918354,87,61.47,-6.81,47.16,30.33,
4,Loa Janan,3,ID,1579918385,48,-0.58,117.09,91.87,4.34,
5,Severo-Kuril'sk,95,RU,1579918391,90,50.68,156.12,17.17,41.03,
6,Sibolga,0,ID,1579918432,59,1.74,98.78,87.08,1.45,
7,Collie,29,AU,1579918443,31,-33.37,116.15,84.99,4.00,
8,Rawson,100,AR,1579918464,47,-43.30,-65.10,70.02,30.87,


In [8]:
#want to find a hotel within 5000 meters of each (lat, lng) in my chosen destinations

#create params dict that will be updated with a hotel name each iteration
params = {"keyword": "Hotel",
          "radius": 5000,
          "key": g_key}

# Use the lat/lng in our hotel_df to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_request = requests.get(base_url, params=params)
    
    #print the hotel_request url, avoid doing for public github repos in order to avoid exposing key
    #print(hotel_request.url)

    # convert to json
    hotel_request = hotel_request.json()
    #print(json.dumps(hotel_request, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_request["results"][0]["name"]
   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
#preview to make sure the dataframe updates
hotel_df       

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Olafsvik,100,IS,1579918001,89,64.89,-23.71,29.14,37.67,Hotel Olafsvik
1,Vestmannaeyjar,100,IS,1579918060,82,63.44,-20.27,39.34,35.28,Hotel Vestmannaeyjar
2,Nanortalik,100,GL,1579918320,75,60.14,-45.24,20.73,31.65,Hotel Kap Farvel
3,Vagur,100,FO,1579918354,87,61.47,-6.81,47.16,30.33,Vestmanna Hostel
4,Loa Janan,3,ID,1579918385,48,-0.58,117.09,91.87,4.34,Hotel Pada Idi
5,Severo-Kuril'sk,95,RU,1579918391,90,50.68,156.12,17.17,41.03,Kak Doma
6,Sibolga,0,ID,1579918432,59,1.74,98.78,87.08,1.45,Prima Indah Hotel
7,Collie,29,AU,1579918443,31,-33.37,116.15,84.99,4.00,Premier Hotel
8,Rawson,100,AR,1579918464,47,-43.30,-65.10,70.02,30.87,Hotel Punta León


In [9]:
#want to drop rows with missing 'Hotel Name' data

#first fill empty 'Hotel Name' cells with NaN
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
#preview
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Olafsvik,100,IS,1579918001,89,64.89,-23.71,29.14,37.67,Hotel Olafsvik
1,Vestmannaeyjar,100,IS,1579918060,82,63.44,-20.27,39.34,35.28,Hotel Vestmannaeyjar
2,Nanortalik,100,GL,1579918320,75,60.14,-45.24,20.73,31.65,Hotel Kap Farvel
3,Vagur,100,FO,1579918354,87,61.47,-6.81,47.16,30.33,Vestmanna Hostel
4,Loa Janan,3,ID,1579918385,48,-0.58,117.09,91.87,4.34,Hotel Pada Idi
5,Severo-Kuril'sk,95,RU,1579918391,90,50.68,156.12,17.17,41.03,Kak Doma
6,Sibolga,0,ID,1579918432,59,1.74,98.78,87.08,1.45,Prima Indah Hotel
7,Collie,29,AU,1579918443,31,-33.37,116.15,84.99,4.00,Premier Hotel
8,Rawson,100,AR,1579918464,47,-43.30,-65.10,70.02,30.87,Hotel Punta León


In [10]:
#find the rows with missing 'Hotel Name' data, for my own reference
hotel_nan = hotel_df[hotel_df.isna().any(axis = 1)]
hotel_nan

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name


In [11]:
#drop the NaN rows and remove newly created 'index column' that's created with reset_index()
hotel_df = hotel_df.dropna().reset_index()
hotel_df = hotel_df[["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", 
                     "Lng", "Max Temp", "Wind Speed", "Hotel Name"]]
#preview
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Olafsvik,100,IS,1579918001,89,64.89,-23.71,29.14,37.67,Hotel Olafsvik
1,Vestmannaeyjar,100,IS,1579918060,82,63.44,-20.27,39.34,35.28,Hotel Vestmannaeyjar
2,Nanortalik,100,GL,1579918320,75,60.14,-45.24,20.73,31.65,Hotel Kap Farvel
3,Vagur,100,FO,1579918354,87,61.47,-6.81,47.16,30.33,Vestmanna Hostel
4,Loa Janan,3,ID,1579918385,48,-0.58,117.09,91.87,4.34,Hotel Pada Idi
5,Severo-Kuril'sk,95,RU,1579918391,90,50.68,156.12,17.17,41.03,Kak Doma
6,Sibolga,0,ID,1579918432,59,1.74,98.78,87.08,1.45,Prima Indah Hotel
7,Collie,29,AU,1579918443,31,-33.37,116.15,84.99,4.00,Premier Hotel
8,Rawson,100,AR,1579918464,47,-43.30,-65.10,70.02,30.87,Hotel Punta León


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
#add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
#add the layer to the map
fig.add_layer(markers)
#display map
fig

Figure(layout=FigureLayout(height='420px'))